In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("Titanic.csv")
df.head()

In [ ]:
# drop some of the features which we will not use in the prediction
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [ ]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [ ]:
inputs

In [ ]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [ ]:
inputs

In [ ]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [ ]:
print(dummies)

In [ ]:
# adding the new female and male columns with integer values in teh table
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [ ]:
inputs

In [ ]:
# droping the sex and male columns as one column is enough to represent sex
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(8)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0
5,3,NaN,8.4583,0
6,1,54.0,51.8625,0
7,3,2.0,21.0750,0


In [ ]:
#. to check if we have any NaN value 
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [ ]:
inputs.Age[:10]   # check the first ten rows of the age column

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [ ]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean()) # to fill the NaN values by the mean of teh ages
inputs.head(10)

,Pclass,Age,Fare,female
0,3,22.000000,7.2500,0
1,1,38.000000,71.2833,1
2,3,26.000000,7.9250,1
3,1,35.000000,53.1000,1
4,3,35.000000,8.0500,0
5,3,29.699118,8.4583,0
6,1,54.000000,51.8625,0
7,3,2.000000,21.0750,0
8,3,27.000000,11.1333,1
9,2,14.000000,30.0708,1


In [ ]:
# split our dataset into train and test and our test size is set 30%
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [ ]:
len(X_train)

623

In [ ]:
len(X_test)

268

In [ ]:
len(inputs)

891

In [ ]:
# importing the Gaussian Naive Bayes formula

from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [ ]:
model.fit(X_train,y_train)

GaussianNB()

In [ ]:
# GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
model.score(X_test,y_test)

0.8246268656716418

In [ ]:
X_test[0:10]

,Pclass,Age,Fare,female
324,3,29.699118,69.5500,0
864,2,24.000000,13.0000,0
11,1,58.000000,26.5500,1
777,3,5.000000,12.4750,1
445,1,4.000000,81.8583,0
269,1,35.000000,135.6333,1
868,3,29.699118,9.5000,0
132,3,47.000000,14.5000,1
244,3,30.000000,7.2250,0
63,3,4.000000,27.9000,0


In [ ]:
y_test[0:10]

324    0
864    0
11     1
777    1
445    1
269    1
868    0
132    0
244    0
63     0
Name: Survived, dtype: int64

In [ ]:
model.predict(X_test[0:10])

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

In [ ]:
model.predict_proba(X_test[:10])

array([[9.58393762e-01, 4.16062383e-02],
       [9.56551457e-01, 4.34485432e-02],
       [9.59134535e-01, 4.08654653e-02],
       [9.59099727e-01, 4.09002732e-02],
       [1.25182799e-02, 9.87481720e-01],
       [9.49559449e-01, 5.04405507e-02],
       [9.60685965e-01, 3.93140349e-02],
       [9.58005973e-01, 4.19940270e-02],
       [9.08789290e-01, 9.12107100e-02],
       [2.96682854e-04, 9.99703317e-01]])

In [ ]:
# calculationg the score using cross validation
from sklearn.model_selection import cross_val_score
result = cross_val_score(GaussianNB(),X_train, y_train, cv=5)
result.mean()

0.7479870967741936

In [ ]:
type(result)

numpy.ndarray

In [ ]:
x = [1,2,3]
x.mean()